<a href="https://colab.research.google.com/github/itimes-digital/mineracao-texto/blob/main/analise_de_texto_para_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
import nltk
import pandas as pd
import re

In [215]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [216]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [217]:
base = pd.read_csv('sample_data/lista_classificada.csv', sep=';', engine='python')

In [218]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",Nao recebi email para validar o cadastro NÃO C...
1,"Cadastro, Login, Senha",Não consigo concluir o cadastro.
2,"Loja C&R, Estoque",Nunca realizei compras pelo app. Pois a forma ...
3,Entrega,"Muito demora pra entregar.Farmácia é urgência,..."
4,Entrega,"Péssimo no que se refere ao item ENTREGA, pois..."


In [219]:
base['Comentario'] = base['Comentario'].map(lambda x: re.sub('[,\.!?-]', '', x))
base['Comentario'] = base['Comentario'].map(lambda x: x.lower())

In [220]:
base.head()

,Categoria,Comentario
0,"Cadastro, Login, Senha",nao recebi email para validar o cadastro não c...
1,"Cadastro, Login, Senha",não consigo concluir o cadastro
2,"Loja C&R, Estoque",nunca realizei compras pelo app pois a forma d...
3,Entrega,muito demora pra entregarfarmácia é urgência q...
4,Entrega,péssimo no que se refere ao item entrega pois ...


In [221]:
stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [222]:
def removeStopWordsAplicandoStemmer(base):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for i in range(len(base)):
        sem_stop = [str(stemmer.stem(p)) for p in base.loc[i,"Comentario"].split() if p not in stopwords]
        frases.append((sem_stop, base.loc[i,"Categoria"]))
    return frases;

In [223]:
frasesComStemming = removeStopWordsAplicandoStemmer(base);
#print(frasesComStemming)

In [224]:
def buscaPalavras(frases):
    todasPalavras = [];
    for (palavras, categoria) in frases:
        todasPalavras.extend(palavras);
    return todasPalavras;

In [225]:
palavras = buscaPalavras(frasesComStemming);
#print(palavras)

In [226]:
def buscaFrequencia(palavras):
    palavras = nltk.FreqDist(palavras);
    return palavras;

In [227]:
frequencia = buscaFrequencia(palavras);
#print(frequencia.most_common(50))

In [228]:
def buscaPalavrasUnicas(frequencia):
    freq = frequencia.keys();
    return freq;

In [229]:
palavrasUnicas = buscaPalavrasUnicas(frequencia);
#print(palavrasUnicas)

In [230]:
def extratorPalavras(documento):
    doc = set(documento);
    caracteristicas = {};
    for palavras in palavrasUnicas:
        caracteristicas['%s' % palavras] = (palavras in doc);
    return caracteristicas;

In [231]:
caracteristicasFrases = extratorPalavras({'am', 'nov', 'dia'});
#print(caracteristicasFrases)

In [232]:
baseCompleta = nltk.classify.apply_features(extratorPalavras, frasesComStemming);
#print(baseCompleta);

In [233]:
classificador = nltk.NaiveBayesClassifier.train(baseCompleta);

In [234]:
print(classificador.show_most_informative_features(10))

Most Informative Features
                  cancel = True           Pedido : Lentid =     78.3 : 1.0
                    cart = True           Erro c : Entreg =     45.4 : 1.0
                     pix = True           Erro P : Lentid =     45.3 : 1.0
                   acess = True           Cadast : Lentid =     33.9 : 1.0
                  adiant = True           Proble : Lentid =     30.5 : 1.0
                   retir = True           Proble : Lentid =     30.5 : 1.0
                     peg = True           Proble : Lentid =     30.5 : 1.0
                  estoqu = True           Loja C : Lentid =     30.5 : 1.0
                  entreg = True           Entreg : Lentid =     30.2 : 1.0
                   códig = True           Erro P : Lentid =     29.0 : 1.0
None


In [235]:
testeFrase = 'Não aparece mais o Retirar na Loja!!! Desisti'

In [236]:
def aplicandoStemmerFrase(frase):
    stemmer = nltk.stem.RSLPStemmer();
    frases = [];
    for (palavras) in frase.split():
        com_stem_sem_stop = [p for p in palavras.split()];
        frases.append(str(stemmer.stem(com_stem_sem_stop[0])));
    return frases;

In [237]:
fraseComStemming = aplicandoStemmerFrase(testeFrase)
print(fraseComStemming)

['não', 'aparec', 'mais', 'o', 'retir', 'na', 'loja!!!', 'desist']


In [238]:
novaFrase = extratorPalavras(fraseComStemming);

In [239]:
print(classificador.classify(novaFrase))

Lentidão, Trava, Crash, Performance


In [240]:
distribuicao = classificador.prob_classify(novaFrase)
for classe in distribuicao.samples():
    print("%s: %f" % (classe, distribuicao.prob(classe)))

Cadastro, Login, Senha: 0.000000
Loja C&R, Estoque: 0.000000
Entrega: 0.004077
Preço, Prazo, Promoção, Stix: 0.000000
Erro PIX: 0.000002
Erro finalização: 0.000000
Cesta: 0.000000
Pedido cancelamento: 0.000000
Erro cartão de crédito: 0.000000
Lentidão, Trava, Crash, Performance: 0.995920
Busca: 0.000000
Problema não detalhado: 0.000000
Novas Funcionalidades: 0.000000
